In [10]:
import numpy as np
from astropy.io import fits
from scipy.spatial import cKDTree
import time

In [12]:
def load_full_hectomap_data():
    """
    Loads the COMPLETE HectoMAP dataset.
    Removes ONLY physically impossible data (e.g. negative redshift) 
    so the code doesn't crash during coordinate conversion.
    """
    # Path from your screenshot
    filepath = '../HectoMAP/HectoMAP_DR16_spec23.fits'
    
    print(f"--- Loading Full Dataset from {filepath} ---")
    
    try:
        hdul = fits.open(filepath)
        data = hdul[1].data
        
        # Load Raw Columns
        ra = data['Z_RA'].astype(np.float64)
        dec = data['Z_DEC'].astype(np.float64)
        z_red = data['Z_TOT_Z'].astype(np.float64)
        
        hdul.close()
        
        total_raw = len(ra)
        print(f"Total raw entries in file: {total_raw}")

        # FILTER: We must remove z <= 0 because they cannot be converted 
        # to distance/velocity (result would be imaginary or infinite).
        # We keep everything else.
        valid_mask = (z_red > 0.0) & (np.isfinite(z_red))
        
        ra = ra[valid_mask]
        dec = dec[valid_mask]
        z_red = z_red[valid_mask]
        
        print(f"Valid galaxies for 3D analysis: {len(ra)}")
        print("(Removed stars/artifacts with z <= 0)")

        # --- CONVERT TO 3D VELOCITY SPACE ---
        c = 3e5  # Speed of light (km/s)
        velocity = z_red * c
        
        ra_rad = np.deg2rad(ra)
        dec_rad = np.deg2rad(dec)

        # X, Y = Transverse Velocity
        # Z = Line-of-Sight Velocity
        x = velocity * np.cos(dec_rad) * np.cos(ra_rad)
        y = velocity * np.cos(dec_rad) * np.sin(ra_rad)
        z = velocity * np.sin(dec_rad)

        return np.vstack((x, y, z)).T

    except FileNotFoundError:
        print(f"CRITICAL ERROR: Could not find file at {filepath}")
        return None

In [14]:
def find_clusters_fof_iterative(points, link_x, link_y, link_z):
    """
    Standard Friends-of-Friends (FoF) algorithm using an ITERATIVE stack 
    (prevents recursion depth errors on full datasets) and KD-Tree for speed.
    """
    start_time = time.time()
    
    print(f"\n--- Starting FoF Clustering on {len(points)} galaxies ---")
    print("Building KD-Tree (Spatial Index)...")
    tree = cKDTree(points)
    
    # Max linking length for initial radial search
    max_link = max(link_x, link_y, link_z)
    
    visited = np.zeros(len(points), dtype=bool)
    clusters = []
    
    # Progress tracking
    print("Running Clustering...")
    
    for i in range(len(points)):
        if visited[i]:
            continue
            
        # Found a new cluster!
        current_cluster_indices = []
        stack = [i]
        visited[i] = True
        
        while stack:
            # Pop the current galaxy index
            curr_idx = stack.pop()
            current_cluster_indices.append(curr_idx)
            
            # 1. Fast Box Search: Find all neighbors within the largest linking length
            # query_ball_point is much faster than checking all N points
            neighbors = tree.query_ball_point(points[curr_idx], r=max_link)
            
            # 2. Precise Ellipsoid Check
            for nb_idx in neighbors:
                if not visited[nb_idx]:
                    # Check exact anisotropic distance
                    p1 = points[curr_idx]
                    p2 = points[nb_idx]
                    
                    dx = (p1[0] - p2[0]) / link_x
                    dy = (p1[1] - p2[1]) / link_y
                    dz = (p1[2] - p2[2]) / link_z
                    
                    dist_sq = dx*dx + dy*dy + dz*dz
                    
                    if dist_sq <= 1.0:
                        visited[nb_idx] = True
                        stack.append(nb_idx)
        
        # Save cluster (storing actual coordinates)
        if len(current_cluster_indices) > 0:
            clusters.append(points[current_cluster_indices])
            
    elapsed = time.time() - start_time
    print(f"DONE. Processed {len(points)} galaxies in {elapsed:.2f} seconds.")
    return clusters

In [16]:
def main():
    # 1. Load ALL data
    points = load_full_hectomap_data()
    
    if points is None:
        return

    # 2. Set Linking Parameters (Standard HectoMAP values)
    # Transverse: ~0.7 - 1.0 Mpc (projected)
    # Line of Sight: ~10x larger to account for "Finger of God" velocity distortion
    lx = 7.0   # Transverse
    ly = 7.0   # Transverse
    lz = 70.0  # Line-of-sight (Velocity)

    print(f"Parameters: Transverse={lx}, LOS={lz}")

    # 3. Run Algorithm
    clusters = find_clusters_fof_iterative(points, lx, ly, lz)

    # 4. Report Results
    n_clusters = len(clusters)
    n_galaxies_clustered = sum(len(c) for c in clusters)
    
    print(f"\n--- RESULTS ---")
    print(f"Total Clusters Found: {n_clusters}")
    print(f"Total Galaxies inside clusters: {n_galaxies_clustered}")
    
    # Filter for significant clusters (e.g., > 10 members)
    large_clusters = [c for c in clusters if len(c) >= 10]
    large_clusters.sort(key=len, reverse=True)
    
    print(f"Number of significant clusters (N >= 10): {len(large_clusters)}")
    
    if large_clusters:
        print("\nTop 5 Massive Clusters:")
        for i in range(min(5, len(large_clusters))):
            print(f"Rank {i+1}: {len(large_clusters[i])} members")

In [18]:
if __name__ == "__main__":
    main()

--- Loading Full Dataset from ../HectoMAP/HectoMAP_DR16_spec23.fits ---
Total raw entries in file: 1148697
Valid galaxies for 3D analysis: 112701
(Removed stars/artifacts with z <= 0)
Parameters: Transverse=7.0, LOS=70.0

--- Starting FoF Clustering on 112701 galaxies ---
Building KD-Tree (Spatial Index)...
Running Clustering...
DONE. Processed 112701 galaxies in 0.74 seconds.

--- RESULTS ---
Total Clusters Found: 111224
Total Galaxies inside clusters: 112701
Number of significant clusters (N >= 10): 3

Top 5 Massive Clusters:
Rank 1: 1160 members
Rank 2: 12 members
Rank 3: 10 members
